In [2]:
import sys
sys.path.extend(["/Users/nelsonlin/Documents/GitHub/RealEstate/src"])
import interpolation as itrp

import censusdata
import us
import pandas as pd
import importlib

importlib.reload(censusdata)

censusdata.search('acs5', 2019, 'concept', 'total population')
#censusdata.printtable(censusdata.censustable('acs5', 2019, 'B23025'))

population_in_occupied_units = "B01003_001E"
median_value = "B25077_001E"
gross_rent_total = "B25031_001E"
median_income = "B19013_001E"

census_columns_to_download = [population_in_occupied_units, median_value, gross_rent_total, median_income]
years_to_download = [2015, 2016, 2017, 2018, 2019]

aggregate_census_df = pd.DataFrame()

for year in years_to_download:
    #or state in us.states.STATES:
    new_data_df = censusdata.download('acs5', year, censusdata.censusgeo([("metropolitan statistical area/micropolitan statistical area", "*")]), census_columns_to_download)
    us_data_df = censusdata.download('acs5', year, censusdata.censusgeo([("us", "*")]), census_columns_to_download)
    new_data_df = pd.concat([new_data_df, us_data_df])

    new_data_df["year"] = year
    aggregate_census_df = aggregate_census_df.append(new_data_df, ignore_index=False)

# cleaning up
new_aggregate_census_df = aggregate_census_df.reset_index()
new_aggregate_census_df.columns = ["RegionName", "population", "median_value", "gross_rent_total", "median_income", "year"]
new_aggregate_census_df.RegionName = new_aggregate_census_df.RegionName.astype(str).apply(lambda x: x[:-98].replace(",", ""))

# recoding names
new_aggregate_census_df.loc[new_aggregate_census_df[new_aggregate_census_df["RegionName"] == ""].index, "RegionName"] = "United States"

to_replace_cities = {
    "Austin-Round Rock TX": "Austin TX",
    "Austin-Round Rock-Georgetown TX": "Austin TX",
    "Cleveland-Elyria OH": "Cleveland OH",
    "Grand Rapids-Wyoming MI": "Grand Rapids MI",
    "Grand Rapids-Kentwood MI" : "Grand Rapids MI",
    "Miami-Fort Lauderdale-Pompano Beach FL": "Miami-Fort Lauderdale FL",
    "Miami-Fort Lauderdale-West Palm Beach FL": "Miami-Fort Lauderdale FL",
    "New Orleans-Metairie LA": "New Orleans LA",
    "Raleigh-Cary NC": "Raleigh NC",
    "San Diego-Carlsbad CA": "San Diego CA",
    "Stockton-Lodi CA": "Stockton CA",
    "Milwaukee-Waukesha WI": "Milwaukee WI",
    "Birmingham-Hoover AL" :  "Birmingham AL",
    "New Haven-Milford CT" : "New Haven CT",
    "Greenville-Anderson SC" : "Greenville SC",
    "Provo-Orem UT" : "Provo UT",
    "San Francisco-Oakland-Hayward CA" : "San Francisco CA",
    "San Francisco-Oakland-Berkeley CA" : "San Francisco CA",
    "San Antonio-New Braunfels TX" : "San Antonio TX",
    "Indianapolis-Carmel-Anderson IN" : "Indianapolis IN",
    "Buffalo-Cheektowaga NY" : "Buffalo NY",
    "Dayton-Kettering OH" : "Dayton OH",
    "Lakeland-Winter Haven FL" : "Lakeland FL",
    "Des Moines-West Des Moines IA" : "Des Moines IA",
    "Dallas-Fort Worth-Arlington TX" : "Dallas-Fort Worth TX",
    "Cape Coral-Fort Myers FL": "Fort Myers FL",
    "Greensboro-High Point NC" : "Greensboro NC",
    "Detroit-Warren-Dearborn MI": "Detroit MI",
    "Riverside-San Bernardino-Ontario CA": "Riverside CA",
    "Phoenix-Mesa-Scottsdale AZ": "Phoenix AZ",
    "Phoenix-Mesa-Chandler AZ": "Phoenix AZ",
    "Seattle-Tacoma-Bellevue WA": "Seattle WA",
    "San Diego-Chula Vista-Carlsbad CA": "San Diego CA",
    "Baltimore-Columbia-Towson MD" : "Baltimore MD",
    "Denver-Aurora-Lakewood CO" : "Denver CO",
    "Sacramento--Roseville--Arden-Arcade CA" : "Sacramento CA",
    "Sacramento-Roseville-Folsom CA" : "Sacramento CA",
    "Orlando-Kissimmee-Sanford FL" : "Orlando FL",
    "Las Vegas-Henderson-Paradise NV" : "Las Vegas NV",
    "Santa Rosa-Petaluma CA" : "Santa Rosa CA",
    "Bridgeport-Stamford-Norwalk CT" : "Stamford CT",
    "Albany-Schenectady-Troy NY" : "Albany NY",
    "Greenville-Anderson-Mauldin SC" : "Greenville SC",
    "McAllen-Edinburg-Mission TX" : "McAllen TX",
    "Little Rock-North Little Rock-Conway AR" : "Little Rock AR",
    "Charleston-North Charleston SC" : "Charleston SC",
    "Ogden-Clearfield UT" : "Ogden UT",
    "Deltona-Daytona Beach-Ormond Beach FL" : "Daytona Beach FL",
    "Durham-Chapel Hill NC" : "Durham NC",
    "Houston-The Woodlands-Sugar Land TX" : "Houston TX",
    "Atlanta-Sandy Springs-Roswell GA" : "Atlanta GA",
    "Atlanta-Sandy Springs-Alpharetta GA" : "Atlanta GA",
    "Hartford-West Hartford-East Hartford CT" : "Hartford CT",
    "Hartford-East Hartford-Middletown CT" : "Hartford CT",
    "Nashville-Davidson--Murfreesboro--Franklin TN" : "Nashville TN",
    "Houston-The Woodlands-Sugar Land TX" : "Houston TX",
    "Tampa-St. Petersburg-Clearwater FL" : "Tampa FL",
    "San Jose-Sunnyvale-Santa Clara CA" : "San Jose CA",
    "Milwaukee-Waukesha-West Allis WI" : "Milwaukee WI",
    "Buffalo-Cheektowaga-Niagara Falls NY" : "Buffalo NY",
    "Oxnard-Thousand Oaks-Ventura CA" : "Ventura CA",
    "New York-Newark-Jersey City NY-NJ-PA" : "New York NY",
    "Chicago-Naperville-Elgin IL-IN-WI" : "Chicago IL",
    "Philadelphia-Camden-Wilmington PA-NJ-DE-MD" : "Philadelphia PA",  
    "Washington-Arlington-Alexandria DC-VA-MD-WV" : "Washington DC",
    "Boston-Cambridge-Newton MA-NH" : "Boston MA",
    "St. Louis MO-IL" : "St. Louis MO",
    "Portland-Vancouver-Hillsboro OR-WA" : "Portland OR",
    "Charlotte-Concord-Gastonia NC-SC" : "Charlotte NC",
    "Cincinnati OH-KY-IN" : "Cincinnati OH",
    "Virginia Beach-Norfolk-Newport News VA-NC" : "Virginia Beach VA",
    "Providence-Warwick RI-MA" : "Providence RI",
    "Memphis TN-MS-AR" : "Memphis TN",
    "Omaha-Council Bluffs NE-IA" : "Omaha NE",
    "Allentown-Bethlehem-Easton PA-NJ" : "Allentown PA",
    "Worcester MA-CT" : "Worcester MA",
    "Kansas City MO-KS" : "Kansas City MO",
    "Louisville/Jefferson County KY-IN" : "Louisville-Jefferson County KY",
    "Minneapolis-St. Paul-Bloomington MN-WI" : "Minneapolis-St Paul MN",
}

for city in to_replace_cities:
    #print (city, to_replace_cities[city])
    new_aggregate_census_df.RegionName = new_aggregate_census_df.RegionName.str.replace(city, to_replace_cities[city])

    
def interpolate_dataframe(new_aggregate_census_df, final_data_year, interpolation_year):
    """
    Requires RegionName
    Interpolates into 2022
    """
    
    interpolated_census = []

    for region in new_aggregate_census_df["RegionName"].unique():

        # select specific region
        test_pd = new_aggregate_census_df[new_aggregate_census_df["RegionName"] == region]

        # add nan so it predicts into future years
        added_rows = pd.DataFrame({"year": range(final_data_year,interpolation_year  + 1)})
        combined_df = pd.concat([test_pd, added_rows]).reset_index(drop=True)
        combined_df = combined_df.set_index("year")

        # Interpolate for each column except the text one
        for col in combined_df.columns:
            if col != "RegionName":
                combined_df[col] = itrp.poly_degree2_trend(combined_df.loc[:, col])["adoption"]

        # Pad nan region name
        combined_df["RegionName"].interpolate(method = "pad", inplace=True)
        interpolated_census.append(combined_df)

    return pd.concat(interpolated_census)

full_interpolated_census_data = interpolate_dataframe(new_aggregate_census_df, final_data_year = 2020, interpolation_year = 2022).to_csv("data/fully_interpolated_census_data_msa.csv")

,B01003_001E,B25077_001E,B25031_001E,B19013_001E,year
"Beaumont-Port Arthur, TX Metro Area: Summary level: 310, metropolitan statistical area/micropolitan statistical area:13140",405695,97700,759.0,45861.0,2015
"Beaver Dam, WI Micro Area: Summary level: 310, metropolitan statistical area/micropolitan statistical area:13180",88547,153100,761.0,53783.0,2015
"Beckley, WV Metro Area: Summary level: 310, metropolitan statistical area/micropolitan statistical area:13220",124027,90100,625.0,38967.0,2015
"Bedford, IN Micro Area: Summary level: 310, metropolitan statistical area/micropolitan statistical area:13260",45814,100500,637.0,45875.0,2015
"Beeville, TX Micro Area: Summary level: 310, metropolitan statistical area/micropolitan statistical area:13300",32659,72900,802.0,42302.0,2015
...,...,...,...,...,...
"Orangeburg, SC Micro Area: Summary level: 310, metropolitan statistical area/micropolitan statistical area:36700",87687,92700,687.0,37955.0,2019
"Orlando-Kissimmee-Sanford, FL Metro Area: Summary level: 310, metropolitan statistical area/micropolitan statistical area:36740",2508970,221600,1210.0,58368.0,2019
"Fort Dodge, IA Micro Area: Summary level: 310, metropolitan statistical area/micropolitan statistical area:22700",36545,99100,642.0,47466.0,2019
"Fort Collins, CO Metro Area: Summary level: 310, metropolitan statistical area/micropolitan statistical area:22660",344786,363800,1297.0,71881.0,2019


In [ ]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

for zillow_region in reframed_true_housing_df.RegionName.unique():
    for census_region in new_aggregate_census_df.RegionName.unique():
        if census_region != zillow_region:
            if "Minneapolis" in census_region and "Minneapolis" in zillow_region:
             #   matches.append([zillow_region, census_region, fuzzy_score])
                print("\"" + census_region + "\" : \""+ zillow_region + "\",")
set(reframed_true_housing_df.RegionName.unique()) - set(new_aggregate_census_df.RegionName.unique())